# Calculating state-by-state implied infection numbers

This notebook tries to compute what the full infection numbers in the past and present likely were/are.

It does so in the past by blending variables for "median days from infection to death" and "infection fatility rate" (IFR) with smoothed death rates. In other words, days_to_death days before date D, there must have been roughly (deaths_on_date_D / IFR) infections to end up with a given number of deaths on date D.

When looking at the most recent days_to_death days, it looks up what percentage of infections were confirmed on the last day calculated in the past, and applies that percentage to the new infections found since then. It normalizes a bit by the amount of testing done on each day to try to handle significant ramping up/down of testing during that time, but the recent projections are admittedly sketchy.

As of December 2021, the data is entirely drawn from two CDC data sets:
* Deaths (https://data.cdc.gov/api/views/r8kw-7aab/rows.csv?accessType=DOWNLOAD)
* Hospitalizations (https://beta.healthdata.gov/api/views/g62h-syeh/rows.csv?accessType=DOWNLOAD)

#### LOCALIZATION REQUIRED:

Look at the `cdc_common.py` file and modify the `download_path()` function to provide a useful place to store some temporary files created in the process of running. It used to expect downloaded files to be there, but now just stores pickle files for caching data.

In [ ]:
%matplotlib inline
import numpy
import pandas
import matplotlib
import matplotlib.pyplot as plt
# import scipy.stats

import cdc_common
from cdc_common import load_data, DOD_META, get_infections_df, download_path, load_hospital_stats

# change the cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pandas.set_option('display.max_rows', 700)
pandas.set_option('display.max_columns', 1000)
pandas.set_option('display.width', 1000)

In [ ]:
EARLIEST_DATE = pandas.Period('2020-03-01', freq='D')

# Set a latest date when the most recent days have garbage (like on or after holidays)
LATEST_DATE = pandas.Period('2020-12-23', freq='D')
LATEST_DATE = None
LATEST_DATE = pandas.Period('2021-12-08', freq='D')

# Set a number of recent days to not display in the graphs for lack of future days to smooth them
NON_DISPLAY_DAYS = 0

In [ ]:
latest_date, meta, all_stats, cdc_stats, hosp_stats = load_data(EARLIEST_DATE, LATEST_DATE)
latest_displayed = latest_date - NON_DISPLAY_DAYS
print(f"Latest date = {str(latest_date)}; latest displayed = {str(latest_displayed)}")

In [ ]:
latest_displayed = latest_date - NON_DISPLAY_DAYS

## Calc Estimated Infections

In [ ]:
# Median number of days between being exposed and developing illness
INCUBATION = 4

# Number of days one is infectious (this isn't actually used yet)
INFECTIOUS = 10

# Median days in between exposure and death
DEATH_LAG = 19

In [ ]:
# Here is where you set variables for IFR assumptions

# Note that this IFR represents a country-wide average on any given day, but the IFRs
# are actually adjusted up/down based on median age and nursing home residents per capita

# This set represents my worst case scenario (in my 95% CI interval)
# Start by setting the inital and final IFRs
IFR_S, IFR_E = 0.013, 0.006
# Then set dates in between by which it linearly scales to various targets
IFR_BREAKS = [['2020-04-30', 0.0095], ['2020-07-31', 0.007], ['2020-09-15', 0.006]]

# This set is my optimistic scenario
IFR_S, IFR_E = 0.01, 0.0025
IFR_BREAKS = [['2020-04-30', 0.0075], ['2020-07-31', 0.0045], ['2020-09-15', 0.0025]]

# This set is a highly optimistic scenario that matches the recent CDC data
IFR_S, IFR_E = 0.009, 0.002
IFR_BREAKS = [['2020-04-30', 0.007], ['2020-07-31', 0.003], ['2020-09-15', 0.002]]

# This is my expected scenario
IFR_S, IFR_E = 0.01, 0.005
IFR_BREAKS = [['2020-04-30', 0.0085], ['2020-07-31', 0.005], ['2020-09-15', 0.004], ['2021-01-15', 0.004],
              ['2021-06-01', 0.0025]]

# This is my expected scenario
IFR_S, IFR_E = 0.014, 0.004
IFR_BREAKS = [['2020-04-30', 0.011], ['2020-07-31', 0.009], ['2020-09-15', 0.007], ['2021-01-15', 0.0055],
              ['2021-06-01', 0.0045]]

# This is my expected scenario
IFR_S, IFR_E = 0.013, 0.003
IFR_BREAKS = [['2020-04-30', 0.01], ['2020-07-31', 0.0085], ['2020-09-15', 0.007], ['2021-01-15', 0.0055],
              ['2021-06-01', 0.004]]

In [ ]:
print("These are the factors applied against IFR state-by-state, based on median age and nursing home numbers:")
IFR_S_S, IFR_E_S = f'{100*IFR_S:.1f}%', f'{100*IFR_E:.2f}%'
infected_states = get_infections_df(all_stats, meta, DEATH_LAG, IFR_S, IFR_E, IFR_BREAKS, INCUBATION, INFECTIOUS)
EST_LINE = str(latest_date - (DEATH_LAG - 1))
print(f"Total infected by {latest_date}: {int(infected_states.NewInf.sum()):,}")
print(f"Vertical line marking recent estimations set at {EST_LINE}")

In [ ]:
infected_states.tail()

## Now for the charts

In [ ]:
# Just nicking off the values we don't want to display here
fazzy = infected_states.reset_index()
fazzy = fazzy[fazzy.Date <= latest_displayed]
fazzy = fazzy.set_index(['ST', 'Date'])
infected_states = fazzy

In [ ]:
fazzle = infected_states.reset_index()[['Date', 'CurrHosp', 'Daily']].groupby('Date').sum()
fazzle.columns = ['Hospitalizations', 'Deaths']
fazzle.loc[:'2020-07-31', 'Hospitalizations'] = numpy.nan
fazzle = fazzle.loc[:str(latest_date), :]
fam = fazzle.plot(
    title=f"Current Hospitalizations vs. Daily Deaths",
    secondary_y='Deaths', figsize=(16,5), ylim=0)
axes = fam.get_figure().get_axes()
for i in range(len(axes)):
    axes[i].set_ylim(0)
# BIDEN_LINE = pandas.Period('2021-01-22', freq='D')
# foo = fam.axvline(BIDEN_LINE, color="red", linestyle="--")

In [ ]:
fazzle

In [ ]:
fazzle.Hospitalizations.max(), fazzle.Deaths.max(), fazzle.Deaths.sum()

In [ ]:
infected_states.columns

In [ ]:
fizzle = infected_states.reset_index()[['Date', 'NewInf', 'Daily']].groupby('Date').sum().copy()
fizzle.columns = ['New Infections', 'Deaths']
# fizzle = fizzle.loc['2020-08-01':, :]
fam = fizzle.plot(
    title=f"New Infections vs. Daily Deaths",
    secondary_y='Deaths', figsize=(16,5), ylim=0)

axes = fam.get_figure().get_axes()
for i in range(len(axes)):
    axes[i].set_ylim(0)

In [ ]:
fizzle.loc['2020-09-05':, :]

In [ ]:
fizzle = infected_states.reset_index()[['Date', 'CurrHosp', 'Daily']].groupby('Date').sum().copy()
fizzle.columns = ['Hospitalizations', 'Deaths']
fizzle.loc[:'2020-07-31', 'Hospitalizations'] = numpy.nan
fam = fizzle.plot(
    title=f"Hospitalizations vs. Daily Deaths",
    secondary_y='Deaths', figsize=(16,5), ylim=0)

axes = fam.get_figure().get_axes()
for i in range(len(axes)):
    axes[i].set_ylim(0)

In [ ]:
# This is where I noodle around to investigate particular states of interest

# This next line lists all 51 (DC included)
st_names = ['SD', 'ND', 'IA', 'TN']
st_names = ['AZ', 'NM', 'PA', 'TX', 'VT',]
st_names = ['AZ', 'PA', 'WV', 'NM', 'MS', 'KS', 'TN', 'SD', 'NV', 'AL',
            'AR', 'RI', 'IL', 'IN', 'SC', 'MI', 'MA', 'CA', 'NJ', 'TX', ]
st_names = ['CA', 'TX', 'PA', 'NY', 'FL', 'AZ', 'IL', 'GA', ]
st_names = ['KS', 'MS', 'AL', 'AZ', 'PA', ]
st_names = ['TX', 'PA', 'GA', 'MA', 'FL', 'NC', 'TN', ]
st_names = ['AK', 'WV', 'ND', 'NM', 'SD']
st_names = ['CA', 'NY', 'IL', 'LA', ]
st_names = ['DC', 'DC',]
st_names = ['CA', 'DC', 'NM', 'MA', 'VA', ]
st_names = ['SD', 'ND', 'IA', 'TN']
st_names = st_names[40:]
st_names = ['NY', 'NJ', 'MA', 'RI', 'IL', 'MI']
st_names = ['ID', 'HI', 'NM', 'AZ', 'NV', 'MS', 'AL', 'GA', 'TX', ]
st_names = ['FL', 'MI']
st_names = list(infected_states.index.get_level_values(0).unique())
num_plots = max(len(st_names), 2)
fig, axes = plt.subplots(num_plots, figsize=(15, 5*num_plots))
for i, st in enumerate(st_names):
    est_lag = DOD_META.get(st, (None, DEATH_LAG))[1]
    est_line = str(latest_date - (est_lag - 1))
    data = infected_states.loc[st, :].reset_index()[['Date', 'CurrHosp', 'DPerM']].copy()
    # data = infected_states.loc[st, :].reset_index()[['Date', 'CurrHosp']].copy()
    data = data[data.Date >= '2020-03-01']
    data.columns = ['Date', 'Hospitalizations', 'Deaths/M']
    data = data.groupby('Date').sum()
    data.loc[:'2020-07-31', 'Hospitalizations'] = numpy.nan
    # data.columns = ['Date', 'Hospitalizations']
    fam = data.plot(
        ax=axes[i], title=st, ylim=0, secondary_y='Deaths/M',
    )
    fam.axvline(est_line, color="red", linestyle="--")

axes = fam.get_figure().get_axes()
for i in range(len(axes)):
    axes[i].set_ylim(0)
    axes[i].set_xlabel(None)

In [ ]:
df = infected_states.reset_index()
# df = df[df.Date < DT]
print(f"{df.NewInf.sum()}, {(df.NewInf.sum()/327_000_000)}")

In [ ]:
# I usually will set this back about 10 days because I don't trust the estimated infections too much
DT = '2021-12-01'
term = 'NIPerM'
divisor = 10000 # 10000 to convert NIPerM to total percentage ever infected
ni = infected_states.reset_index()[['ST', 'Date', term]].copy()
ni = ni[ni.Date < DT].copy()
ni = (ni.groupby('ST').sum()[term].sort_values(ascending=False) / divisor)
# for v in ni.sort_index().values:
#     print(v/100)
ni

In [ ]:
infected_states.Daily.sum()

In [ ]:
raise ValueError()

In [ ]:
earliest_date = pandas.Period(str(EARLIEST_DATE), freq='D')
latest_date = LATEST_DATE
if not latest_date:
    latest_date = (datetime.datetime.now() - datetime.timedelta(hours=19)).date()
latest_date = pandas.Period(str(latest_date), freq='D')
all_dates = pandas.period_range(start=earliest_date, end=latest_date, freq='D')

uri = download_path("cdc_hospitalization_data.csv")
hosp_stats = load_hospital_stats(uri, meta, all_dates)
hosp_stats.reset_index().groupby('Date').sum().tail(20)

In [ ]:
spaz = infected_states.reset_index()[['Date', 'Daily']].groupby('Date').sum().copy()
spaz.loc['2021-01-03':'2021-02-03'].sum()

In [ ]:
infected_states.columns

In [ ]:
inf = infected_states.loc['MA'].NewInf.sum()
dead = infected_states.loc['MA'].Daily.sum()
pop = infected_states.loc['MA'].Pop.mean() * 1000000
vacc = (4_079_138 + 240_703)
inf, dead, vacc, pop, (100 * (dead / inf)), (vacc / pop), (inf - dead) / pop, vacc / pop

In [ ]:
((inf - dead) + vacc - (vacc * ((vacc / pop) * 0.8))) / pop, (vacc / pop) * 0.8

In [ ]:
hosp_stats.reset_index().groupby('Date').sum()[['CurrHosp']].sort_index()

In [ ]:
# list(infected_states.index.get_level_values(0).unique())

In [ ]:
num_plots = max(len(DOD_META), 2)
fig, axes = plt.subplots(num_plots, figsize=(15, 5*num_plots))
for i, (st, __, ignore_days, __) in enumerate(DOD_META):
    data = infected_states.loc[st, :].reset_index()[['Date', 'CurrHosp', 'DPerM']].copy()
    data = data[data.Date >= '2020-10-01']
    data.columns = ['Date', 'Hospitalizations', 'Deaths/M']
    fam = data.groupby('Date').sum().plot(
        ax=axes[i], title=st, ylim=0, secondary_y='Deaths/M',
    )
    est_line = str(latest_date - ignore_days)
    fam.axvline(est_line, color="red", linestyle="--")

axes = fam.get_figure().get_axes()
for i in range(len(axes)):
    axes[i].set_ylim(0)
    axes[i].set_xlabel(None)

In [ ]:
st_names = ['AR', 'CO', 'DC', 'DE', 'HI', 'IA', 'ID',
            'KS', 'KY', 'MD', 'ME', 'MN', 'MT', 'NE', 'NH', 'NM',
            'OK', 'OR', 'UT', 'VT', 'WA', 'WI', 'WV', 'WY']
st_names = ['HI', 'MT', ]
st_names = ['DE', 'KS', ]
num_plots = max(len(st_names), 2)
fig, axes = plt.subplots(num_plots, figsize=(15, 5*num_plots))
for i, st in enumerate(st_names):
    data = infected_states.loc[st, :].reset_index()[['Date', 'Hospital5', 'DPerM']].copy()
    data = data[data.Date >= '2020-11-01']
    data.columns = ['Date', 'Hospitalizations', 'Deaths/M']
    fam = data.groupby('Date').sum().plot(
        ax=axes[i], title=st, ylim=0, secondary_y='Deaths/M',
    )
    fam.axvline(EST_LINE, color="red", linestyle="--")

axes = fam.get_figure().get_axes()
for i in range(len(axes)):
    axes[i].set_ylim(0)

In [ ]:
df = infected_states.reset_index()
# df = df[df.Date < DT]
print(f"{(df.NewInf.sum()/327_000_000)}")

In [ ]:
# I usually will set this back about 10 days because I don't trust the estimated infections too much
DT = '2021-02-20'
term = 'NIPerM'
divisor = 10000 # 10000 to convert NIPerM to total percentage ever infected
ni = infected_states.reset_index()[['ST', 'Date', term]].copy()
ni = ni[ni.Date < DT].copy()
ni = (ni.groupby('ST').sum()[term].sort_values(ascending=False) / divisor)
# for v in ni.sort_index().values:
#     print(v/100)
ni

In [ ]:
# Stopping the processing of this notebook
raise ValueError()

## Detritus

In [ ]:
infected_states.reset_index().columns